In [2]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [3]:
from datasets import load_dataset

corpus = load_dataset("allenai/scifact","corpus")

Generating train split:   0%|          | 0/5183 [00:00<?, ? examples/s]

In [4]:
dataset = load_dataset("allenai/scifact","claims")

Generating train split:   0%|          | 0/1261 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/300 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/450 [00:00<?, ? examples/s]

In [33]:
dataset['train'][21]

{'id': 28,
 'claim': 'A T helper 2 cell (Th2) environment impedes disease development in patients with systemic lupus erythematosus (SLE).',
 'evidence_doc_id': '12670680',
 'evidence_label': 'CONTRADICT',
 'evidence_sentences': [1],
 'cited_doc_ids': [12670680]}

In [6]:
corpus['train'][5]

{'doc_id': 33370,
 'title': 'Targeting A20 Decreases Glioma Stem Cell Survival and Tumor Growth',
 'abstract': ['Glioblastomas are deadly cancers that display a functional cellular hierarchy maintained by self-renewing glioblastoma stem cells (GSCs).',
  'GSCs are regulated by molecular pathways distinct from the bulk tumor that may be useful therapeutic targets.',
  'We determined that A20 (TNFAIP3), a regulator of cell survival and the NF-kappaB pathway, is overexpressed in GSCs relative to non-stem glioblastoma cells at both the mRNA and protein levels.',
  'To determine the functional significance of A20 in GSCs, we targeted A20 expression with lentiviral-mediated delivery of short hairpin RNA (shRNA).',
  'Inhibiting A20 expression decreased GSC growth and survival through mechanisms associated with decreased cell-cycle progression and decreased phosphorylation of p65/RelA. Elevated levels of A20 in GSCs contributed to apoptotic resistance: GSCs were less susceptible to TNFalpha-i

In [7]:
corpus2 = {}
for doc in corpus['train']:
  title = doc['title']
  abstract = ''
  for sent in doc['abstract']:
    abstract = abstract + sent + ' '
  corpus2[doc['doc_id']] = {'title':title,'abstract':abstract}

In [36]:
import pandas as pd
new_dataset = pd.DataFrame(columns=['claim','abstract','label'])
cnt = 0
for item in dataset['train']:
  doc_id = item['cited_doc_ids'][0]
  if item['evidence_label'] == '':
    label = "NO_EVIDENCE"
  if item['evidence_label'] == 'SUPPORT':
    label = 'SUPPORT'
  if item['evidence_label'] == 'CONTRADICT':
    label = 'CONTRADICT'
  claim = item['claim']
  abstract = corpus2[doc_id]['title']+'\n'+corpus2[doc_id]['abstract']
  new_dataset.loc[cnt] = pd.Series({'claim':claim, 'abstract':abstract, 'label':label})
  cnt = cnt + 1




In [41]:
new_dataset

,claim,abstract,label
0,0-dimensional biomaterials lack inductive prop...,New opportunities: the use of nanotechnologies...,NO_EVIDENCE
1,1 in 5 million in UK have abnormal PrP positiv...,Prevalent abnormal prion protein in human appe...,CONTRADICT
2,1-1% of colorectal cancer patients are diagnos...,Relation between Medicare screening reimbursem...,NO_EVIDENCE
3,10% of sudden infant death syndrome (SIDS) dea...,Sudden infant death syndrome.\nDespite decline...,NO_EVIDENCE
4,32% of liver transplantation programs required...,Liver transplantation and opioid dependence.\n...,SUPPORT
...,...,...,...
1256,siRNA knockdown of A20 slows tumor progression...,Targeting A20 Decreases Glioma Stem Cell Survi...,SUPPORT
1257,siRNA knockdown of A20 slows tumor progression...,Targeting A20 Decreases Glioma Stem Cell Survi...,SUPPORT
1258,taa1 tar1 tar2 triple mutants in Arabidopsis d...,TAA1-Mediated Auxin Biosynthesis Is Essential ...,NO_EVIDENCE
1259,β-sheet opening occurs during pleurotolysin po...,Conformational Changes during Pore Formation b...,SUPPORT
